### BEGIN: Confusion matrix for prop. method

In [ ]:
#########################################
#########################################
#########################################
#########################################


import pandas as pd
import math
import geopandas as gpd
import numpy as np

model = pm.database.sqlite.load('F:\\AdelsCode\\Data\\simulation data\\AllGPS5WithCovariates.sqlite')
s5 = model.trace('states').gettrace()[:]
s5 = np.median(s5,axis=0)

dist = []
timeDiff = []
WithStat = []

df = gpd.read_file("F:\\AdelsCode\\Data\\AllGPS5WithCovariates.shp")
# df = df.sample(frac=0.6, replace=True)


df["nearTube"] = df["rail"]
df["nearBus"] = df["bus"]
df["div"] = df["SpeedMS"]
df = df[df["div"] > 0]

df["time"] = pd.to_datetime(df["fixTime"], format="%d/%m/%Y %H:%M")
# df = df[(df['time'] > '13/05/2010')]

df = df.sort("time")
# df = df.iloc[0:100]
df = df.reset_index()

def distance(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return (c * r) * 1000

for i in range(0, df.shape[0]-1):
    lon2 = df.iloc[i]['latitude']
    lon1 = df.iloc[i+1]['latitude']
    lat2 = df.iloc[i]['longitude']
    lat1 = df.iloc[i+1]['longitude']
    dist.append(distance(lat1, lon1, lat2, lon2))
    timeDiff.append((df["time"][i+1]-df["time"][i]).total_seconds())
    WithStat.append(df.iloc[i+1]['WithStat'])
   
    #ids.append(df["id"][i])
   
df2 = pd.DataFrame(dist, columns = ['dist'])
i = 0
for x in timeDiff:
    if x<0:
        timeDiff[i]=0
    i = i+1
   
   
df2['time'] = timeDiff
df2['div'] = df2['dist']/df2['time']
df2['fixTime'] = df["fixTime"]
df2['nearBus'] = df["nearBus"]
df2['nearTube'] = df["nearTube"]
df2['IMDS10'] = df["IMDS10"]
df2['WithStat'] = WithStat
df2 = df2[df2['WithStat'] != 'cycle']

df2 = df2[np.isfinite(df['div'])]
df2 = df2.reset_index()

data5 = df2
data5 = data5[["nearBus","nearTube","IMDS10","div", "WithStat"]]
# data.to_csv("F:\\AdelsCode\\Data\\test.csv")
data5 = data5[data["div"] > 0]

data5.reset_index(inplace=True)

data5.set_value(data5[data5["WithStat"] == "stationary"].index,"WithStat",  0)
data5.set_value(data5[data5["WithStat"] == "walk"].index,"WithStat",  1)
data5.set_value(data5[data5["WithStat"] == "bus"].index,"WithStat",  2)
data5.set_value(data5[data5["WithStat"] == "train"].index,"WithStat",  3)
data5.set_value(data5[data5["WithStat"] == "tube"].index,"WithStat",  3)
data5[pd.isnull(data5.WithStat)] = 0

j5 = confusion_matrix(s5, (data5[["WithStat"]].values.flatten()).astype(int))
print j5
print "accuracy: ", float(np.trace(j5))/j5.sum()



######################################


### END: Confusion matrix for prop. method